### 1. Utilitários

In [1]:
import pandas as pd
import os

In [2]:
# Lista de 3 valores:
# Caminho dos arquivo no diretório. (str)
# Nome para Dataframe. (str)
# Nome para subtituir coluna "Total" de cada arquivo. (str)
dados = [
    (os.path.join("..", "Área de Culturas Permanentes (hectares)", "Área colhida permanente 1999-2020.csv"), "df_permanente", "Culturas P. Colhidas (hec)"),
    (os.path.join("..", "Producao-Milhos", "Produção - milho(R$).xls"), "df_milho", "Producao de Milho (R$)"),
    (os.path.join("..", "Producao-Soja", "Produção - soja (R$).csv"), "df_soja", "Producao de Soja (R$)"),
    (os.path.join("..", "Despesa Orçamentária (R$)", "Despesa por função - gestão ambiental, agricultura e organização agrária - empenhada - estadual.xls"), "df_despesa", "Despesa Orçamentária (R$)"),
    (os.path.join("..", "Focos de Incendio", "firefoci.csv"), "df_incendio", "Focos de Incêndio"),
    (os.path.join("..", "Área de Culturas Temporárias (hectares)", "Área colhida temporária 1999-2020.csv"), "df_temporaria", "Culturas T. Colhidas (hec)"),
    (os.path.join("..", "Área Plantada ou Colhida (hectares)", "Área plantada ou destinada à colheita - total.xls"), "df_plantada_colhida", "Área Plantada ou Colhida (hec)"),
    (os.path.join("..", "PIB", "PIB 1999-2020.csv"), "df_pib", "PIB Estadual"),
    (os.path.join("..", "Valor da Produção Agrícola (R$)", "Valor de Produção 1999-2020.csv"), "df_producao", "PIB Agrícula Estadual") 
]

# Lista dos estados que queremos filtrar
estados = ["Acre", "Amazonas", "Amapá", "Maranhão", "Pará", "Roraima", "Tocantins", "Rondônia", "Mato Grosso"] 

# Lista de anos como strings
anos_str = [str(year) for year in range(1999, 2021)]

### 2. Pré-processamento

O objetivo final é unir os valores anuais de 1999 a 2020 de cada variável estadual dos 9 estados da região em uma tabela única, para futura implementação de um algoritmo de machine learning.

In [3]:
for arquivo, nome_df, coluna in dados:
    try:
        # Ler arquivos .csv com separador ',', se não, ler arquivo .xls
        try:
            df = pd.read_csv(arquivo, sep=",")
        except:
            df = pd.read_excel(arquivo)

        # Normalizar os nomes das colunas (remover espaços extras)
        df.columns = df.columns.str.strip()

        # Configurar Pandas para exibir números inteiros completos
        pd.options.display.float_format = '{:.2f}'.format
        
        # Remover colunas "Código" e "Sigla", se existirem
        colunas_para_remover = [col for col in ["Codigo", "Código", "Sigla"] if col in df.columns]
        if colunas_para_remover:
            df = df.drop(columns=colunas_para_remover)
            print(f"🗑️ Colunas removidas de {nome_df}: {colunas_para_remover}")

        # Filtrar os estados desejados
        if "Estado" in df.columns:
            df = df[df["Estado"].isin(estados)]
            print(f"🌎 Filtrados apenas os estados desejados em {nome_df}")

        # Armazenar na variável correta
        globals()[nome_df] = df
        print(f"✔ {arquivo} carregado como {nome_df}")
        
    except Exception as e:
        print(f"Erro ao ler {arquivo}: {e}")

# Exibir quais DataFrames foram carregados
print("\n📂 DataFrames disponíveis:", list(nome_df for _, nome_df, _ in dados))

🗑️ Colunas removidas de df_permanente: ['Codigo', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_permanente
✔ ..\Área de Culturas Permanentes (hectares)\Área colhida permanente 1999-2020.csv carregado como df_permanente
🗑️ Colunas removidas de df_milho: ['Codigo', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_milho
✔ ..\Producao-Milhos\Produção - milho(R$).xls carregado como df_milho
🗑️ Colunas removidas de df_soja: ['Sigla']
🌎 Filtrados apenas os estados desejados em df_soja
✔ ..\Producao-Soja\Produção - soja (R$).csv carregado como df_soja
🗑️ Colunas removidas de df_despesa: ['Codigo', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_despesa
✔ ..\Despesa Orçamentária (R$)\Despesa por função - gestão ambiental, agricultura e organização agrária - empenhada - estadual.xls carregado como df_despesa
✔ ..\Focos de Incendio\firefoci.csv carregado como df_incendio
🗑️ Colunas removidas de df_temporaria: ['Código', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_

##### 3. Transformação formato Wide para Long

Os DataFrames estão no formato longo, com 22 colunas representando os anos de 1999 a 2020, exceto no Data frame "Focos de incêndio", em que as colunas representam os 9 estados.
A função .melt() permite transformar essas colunas em linhas, facilitando a análise e manipulação dos dados.

In [4]:
df_incendio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Ano          22 non-null     int64  
 1   Acre         22 non-null     float64
 2   Amapá        22 non-null     float64
 3   Amazonas     22 non-null     float64
 4   Maranhão     22 non-null     float64
 5   Mato Grosso  22 non-null     float64
 6   Pará         22 non-null     float64
 7   Rondônia     22 non-null     float64
 8   Roraima      22 non-null     float64
 9   Tocantins    22 non-null     float64
dtypes: float64(9), int64(1)
memory usage: 1.8 KB


In [5]:
for arquivo, nome_df, coluna in dados:
    df = globals()[nome_df]
    try:
        df = df.melt(id_vars=['Estado'], value_vars=anos_str, var_name='Ano', value_name= f'{coluna}')
    except:
        #alterações df_incendio
        df["Ano"] = df['Ano'].astype(str)
        df = df.melt(id_vars=["Ano"], value_vars=estados, var_name="Estado", value_name= f'{coluna}')
    globals()[nome_df] = df

In [6]:
df_incendio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ano                198 non-null    object 
 1   Estado             198 non-null    object 
 2   Focos de Incêndio  198 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.8+ KB


##### 3. Merge do DataFrame final

In [7]:
df_geral = df_permanente.merge(df_milho, how='outer', on=["Ano", "Estado"])\
                            .merge(df_soja, how='outer', on=["Ano", "Estado"] )\
                            .merge(df_despesa, how='outer', on=["Ano", "Estado"])\
                            .merge(df_temporaria, how='outer', on=["Ano", "Estado"])\
                            .merge(df_plantada_colhida, how='outer', on=["Ano", "Estado"])\
                            .merge(df_pib, how='outer', on=["Ano", "Estado"])\
                            .merge(df_producao, how='outer', on=["Ano", "Estado"])\
                            .merge(df_incendio, how='outer', on=["Ano", "Estado"])
df_geral

,Estado,Ano,Culturas P. Colhidas (hec),Producao de Milho (R$),Producao de Soja (R$),Despesa Orçamentária (R$),Culturas T. Colhidas (hec),Área Plantada ou Colhida (hec),PIB Estadual,PIB Agrícula Estadual,Focos de Incêndio
0,Acre,1999,9864,18904.94,135.89,16480351.00,81922,91787,1866542.32,280900.59,347.00
1,Amapá,1999,1283,663.59,NaN,2869221.75,6903,8235,2128307.03,54019.05,101.00
2,Amazonas,1999,61397,12572.44,875.29,17685970.41,150055,213964,13933920.08,623336.20,1048.00
3,Maranhão,1999,28896,96560.50,243966.36,15239850.98,1130797,1180275,10308256.16,1174349.20,8712.00
4,Mato Grosso,1999,91480,287455.01,3470093.09,43232165.00,4259560,4445242,12364904.96,6462377.66,44307.00
...,...,...,...,...,...,...,...,...,...,...,...
193,Mato Grosso,2020,37589,10090285.97,22939332.03,555308252.18,17149919,17188641,178649564.00,41799931.48,47708.00
194,Pará,2020,651600,346956.98,1428548.90,380705994.45,1310866,1968093,215935604.00,7981858.68,38603.00
195,Rondônia,2020,83545,396740.36,884772.42,210685707.64,721826,813919,51598741.00,2196163.25,11145.00
196,Roraima,2020,10007,47137.95,162262.75,67296396.75,84045,94052,16024276.00,422452.65,1930.00


##### 4. Normalização por área

Ajustar os valores de cada registro de acordo com a extensão territorial de seu respectivo estado

In [ ]:
arquivo_area = os.path.join("Extensão Territorial", "IBGE_area_geografica_98-22.csv")
df_area = pd.read_csv(arquivo_area, sep=";", skiprows=1)

In [9]:
df_area

,Sigla,Código,Estado,1998,2000,2010,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 16
0,AC,12,Acre,"153149,9","152521,9","164123,04","164123,739","164123,739","164123,712","164123,737","164123,737","164123,738","164123,964","164173,431","164173,431","164173,431",NaN
1,AL,27,Alagoas,"27933,1","27818,9","27778,506","27774,993","27848,003","27848,158","27848,14","27848,14","27843,295","27843,295","27830,656","27830,657","27830,657",NaN
2,AM,13,Amazonas,"1577820,2","1570947,1","1559159,148","1559148,889","1559148,89","1559149,074","1559146,876","1559146,876","1559168,117","1559167,889","1559167,878","1559167,879","1559255,883",NaN
3,AP,16,Amapá,"143453,7","142815,8","142828,521","142828,52","142828,52","142828,523","142828,521","142828,521","142470,762","142470,762","142470,762","142470,762","142470,762",NaN
4,BA,29,Bahia,"567295,3","564272,3","564733,177","564733,081","564733,081","564732,642","564732,45","564732,45","564722,611","564760,427","564760,427","564760,427","564760,426",NaN
5,CE,23,Ceará,"146348,3","145712,3","148920,472","148886,308","148886,308","148887,632","148887,633","148887,633","148894,757","148894,441","148894,442","148894,441","148894,441",NaN
6,DF,53,Distrito Federal,"5822,1","5801,9","5779,999","5779,999","5779,999","5779,999","5779,997","5779,997","5760,783","5760,783","5760,784","5760,784","5760,784",NaN
7,ES,32,Espírito Santo,"46184,1","46047,2","46095,583","46095,299","46096,925","46089,39","46086,907","46086,907","46074,444","46074,447","46074,447","46074,447","46074,447",NaN
8,GO,52,Goiás,"341289,5","340118,8","340111,783","340111,376","340111,376","340110,385","340106,492","340106,492","340125,715","340203,329","340242,854","340242,859","340242,858",NaN
9,MA,21,Maranhão,"333365,6","331918,6","331937,45","331936,948","331936,948","331936,955","331936,949","331936,949","329642,17","329642,182","329651,495","329651,495","329651,495",NaN


In [ ]:
#Seleciona os estados de interesse
df_area = df_area.query("@estados in Estado")

#Seleciona os dados mais recentes (2022)
df_area22 = df_area[["Estado", "Area"]]
df_area22

,Estado,2022
0,Acre,"164173,431"
2,Amazonas,"1559255,883"
3,Amapá,"142470,762"
9,Maranhão,"329651,495"
12,Mato Grosso,"903208,362"
13,Pará,"1240882,471"
20,Rondônia,"237754,173"
21,Roraima,"223644,527"
26,Tocantins,"277423,631"


teste

In [ ]:
df_com_area = df_geral.merge(df_area22, how='outer', on=["Estado"])
df_com_area

,Estado,Ano,Culturas P. Colhidas (hec),Producao de Milho (R$),Producao de Soja (R$),Despesa Orçamentária (R$),Culturas T. Colhidas (hec),Área Plantada ou Colhida (hec),PIB Estadual,PIB Agrícula Estadual,Focos de Incêndio,2022
0,Acre,1999,9864,18904.94,135.89,16480351.00,81922,91787,1866542.32,280900.59,347.00,"164173,431"
1,Acre,2000,11194,24988.62,NaN,28832578.47,93973,105167,2154284.48,313604.57,430.00,"164173,431"
2,Acre,2001,10652,20954.80,NaN,23410698.78,91928,102604,2475169.50,239926.84,829.00,"164173,431"
3,Acre,2002,13487,29299.78,NaN,41052040.32,90418,113139,2971301.00,269826.59,6739.00,"164173,431"
4,Acre,2003,14160,35786.63,NaN,38074672.00,95655,113362,3377234.00,305032.27,10523.00,"164173,431"
...,...,...,...,...,...,...,...,...,...,...,...,...
193,Tocantins,2016,4572,205070.37,1220868.41,218469173.36,1195553,1232549,31584816.00,2109560.61,14494.00,"277423,631"
194,Tocantins,2017,4083,229747.15,1397605.34,227070044.04,1278379,1282545,34108132.00,2274246.75,15673.00,"277423,631"
195,Tocantins,2018,13046,232523.41,1530335.31,231303102.81,1348373,1361447,35666183.00,2512067.49,8033.00,"277423,631"
196,Tocantins,2019,13704,298800.31,1575215.24,253891184.06,1413202,1427218,39355941.00,2678345.23,13625.00,"277423,631"


In [ ]:
df_geral.to_csv("tabela_geral", index = False, encoding="utf-8")
path = os.getcwd()

print("📂 Dataframe salvo em: ", path) 